In [128]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
import seaborn
import re

In [129]:
df = pd.read_csv('data.csv', parse_dates=True, date_parser=dateparse, dtype = {'Id':'string', 'Title': 'string', 'Company': 'string', 'Date': 'string', 'Location': 'string', 'Area': 'string', 'Classification': 'string', 'SubClassification': 'string', 'Requirement': 'string', 'FullDescription': 'string', 'LowestSalary': 'int', 'HighestSalary': 'int', 'JobType': 'string'})
df['Id'] = df['Id'].str.extract('(^[0-9]+)').astype(int)
df['Date'] = pd.to_datetime(df['Date'])

In [130]:
print(df.head())
print('\n', df.dtypes)

         Id                                              Title  \
0  37404348                           Casual Stock Replenisher   
1  37404337                           Casual Stock Replenisher   
2  37404356  RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...   
3  37404330                              Team member - Belrose   
4  37404308  Business Banking Contact Centre Specialist, Ni...   

                                          Company                      Date  \
0                                     Aldi Stores 2018-10-07 00:00:00+00:00   
1                                     Aldi Stores 2018-10-07 00:00:00+00:00   
2                             LB Creative Pty Ltd 2018-10-07 00:00:00+00:00   
3                          Anaconda Group Pty Ltd 2018-10-07 00:00:00+00:00   
4  Commonwealth Bank - Business & Private Banking 2018-10-07 00:00:00+00:00   

                  Location                         Area  \
0                   Sydney  North West & Hills District   
1    Richm